## 1. A LOOK IN THE DATA 

In [49]:
! pip install gensim

In [124]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps=PorterStemmer()
lemmatizer=WordNetLemmatizer()
import gensim
from gensim.models import Word2Vec
import time 
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
data=pd.read_csv('C:/Users/sarab/Downloads/cleaneddata.csv')


In [51]:
del data['Unnamed: 0']

In [52]:
data

,title,summary,instructions,ingredients,ingredient types,diets,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,healthScore,pricePerServing,readyInMinutes,servings
0,orange fig teacake with caramel glaze,orange fig teacake with caramel glaze is a veg...,you will need a 9 springform pan or a cake ...,ap flour; baking powder; cardamom; eggs; fresh...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,True,False,False,False,False,False,False,False,3.0,75.55,45,10
1,poached eggs on a bed of fried mushrooms and c...,poached eggs on a bed of fried mushrooms and c...,in a frying pan heat up oil then add mushroom...,bread; butter; eggs; eggs; mushrooms; oil; sal...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,True,False,False,False,False,False,False,False,15.0,147.70,45,2
2,pandan chiffon cake,for 26 cents per serving this recipe covers ...,preheat the oven to 170c blend the pandan le...,all purpose flour; bay leaves; coconut milk; c...,Ethnic Foods Produce Spices and Seasonings Bev...,dairy free; lacto ovo vegetarian,True,False,False,True,False,False,False,False,1.0,26.06,45,9
3,pork chop with honey mustard and apples,pork chop with honey mustard and apples might...,pre heat your oven to 200c 400f line a roa...,apples; dijon mustard; garlic cloves; honey; j...,Meat Spices and Seasonings Condiments Oil Vine...,gluten free; dairy free; paleolithic; primal,False,False,True,True,False,False,False,False,17.0,242.23,45,4
4,beet gnocchi with steak and brown butter sauce,the recipe beet gnocchi with steak and brown b...,cooking beets heat oven to 400 degrees wash be...,gnocchi; beets; olive oil; s p; goat cheese; r...,Produce Spices and Seasonings Meat Spices and ...,NaN,False,False,False,False,False,False,False,False,12.0,417.69,45,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,yakitori glaze,yakitori glaze might be just the japanese reci...,combine all ingredients in a small saucepan a...,butter; egg yolks; flour; orange zest; poppy s...,Milk Eggs Other Dairy Spices and Seasonings Ba...,lacto ovo vegetarian,True,True,True,True,False,False,False,False,1.0,193.70,45,4
4301,pan seared honey glazed salmon with collard gr...,pan seared honey glazed salmon with collard gr...,instructions collard greens using a non stick ...,unbaked pie crust; shredded cheddar cheese; ja...,Milk Eggs Other Dairy Meat Spices and Seasonin...,NaN,False,False,True,True,True,False,False,False,63.0,493.63,30,2
4302,white chocolate cheesecake with raspberries,the recipe white chocolate cheesecake with ras...,to make the crumb crust in a small bowl combin...,egg yolks; grand marnier; light brown sugar; o...,Beverages Milk Eggs Other Dairy Alcoholic Beve...,gluten free; lacto ovo vegetarian,False,False,False,False,False,False,False,False,2.0,139.70,45,16
4303,fish pie with fresh and smoked salmon,this recipe makes 2 servings with 710 calories...,peel potatoes and cut into chunks cook in boi...,cashew; cumin; eggplant; flour; garam masala; ...,Beverages Spices and Seasonings Canned and Jar...,dairy free; lacto ovo vegetarian; vegan,False,False,True,False,True,False,False,False,90.0,1013.56,45,2


Avoid null values

In [53]:
data['diets'] = data['diets'].fillna('')

In [54]:
data.describe()

,healthScore,pricePerServing,readyInMinutes,servings
count,4305.000000,4305.000000,4305.000000,4305.000000
mean,19.432985,195.611954,56.468293,8.331010
std,23.292771,249.970980,95.761049,10.626728
min,0.000000,0.930000,2.000000,1.000000
25%,3.000000,70.100000,45.000000,4.000000
50%,11.000000,139.700000,45.000000,6.000000
75%,27.000000,250.630000,45.000000,8.000000
max,100.000000,8230.170000,3595.000000,250.000000


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4305 entries, 0 to 4304
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             4305 non-null   object 
 1   summary           4300 non-null   object 
 2   instructions      4104 non-null   object 
 3   ingredients       4305 non-null   object 
 4   ingredient types  4305 non-null   object 
 5   diets             4305 non-null   object 
 6   vegetarian        4305 non-null   bool   
 7   vegan             4305 non-null   bool   
 8   glutenFree        4305 non-null   bool   
 9   dairyFree         4305 non-null   bool   
 10  veryHealthy       4305 non-null   bool   
 11  cheap             4305 non-null   bool   
 12  veryPopular       4305 non-null   bool   
 13  sustainable       4305 non-null   bool   
 14  healthScore       4305 non-null   float64
 15  pricePerServing   4305 non-null   float64
 16  readyInMinutes    4305 non-null   int64  


healthScore was actually int values 

In [58]:
data.healthScore = data.healthScore.astype(int)


The other float one is price, can be round , no such a big diference on the meaning 

First selection of variables to try de model 

In [59]:
tex = data.iloc[:,[0,-4,-2,-1]]
tex

,title,healthScore,readyInMinutes,servings
0,orange fig teacake with caramel glaze,3,45,10
1,poached eggs on a bed of fried mushrooms and c...,15,45,2
2,pandan chiffon cake,1,45,9
3,pork chop with honey mustard and apples,17,45,4
4,beet gnocchi with steak and brown butter sauce,12,45,4
...,...,...,...,...
4300,yakitori glaze,1,45,4
4301,pan seared honey glazed salmon with collard gr...,63,30,2
4302,white chocolate cheesecake with raspberries,2,45,16
4303,fish pie with fresh and smoked salmon,90,45,2


# WORD2VEC

## Text Processing 

In [60]:
! pip install stop-words

In [61]:
stop_words=set(stopwords.words('english'))
lista=[]
for fila in tex["instructions"]:
    fil=fila.lower()
    #delete links
    sinwebs=re.sub(r"https\S+|https\S+|www\S+",'',fil,flags=re.MULTILINE)
    #tokenize
    tok=word_tokenize(sinwebs)
    filaa=[]
    for pal in tok:
        #delete stopwords
        if pal not in stop_words:
            if pal.isalnum():
                #stemize
                z=ps.stem(pal)
                #lematize
                zz=lemmatizer.lemmatize(z)
                filaa.append(zz)
    lista.append(filaa) 
tex['cols']=lista

<ipython-input-61-a7064eb2ddad>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tex['cols']=lista


In [62]:
tex

,title,healthScore,readyInMinutes,servings,cols
0,orange fig teacake with caramel glaze,3,45,10,"[orang, fig, teacak, caramel, glaze]"
1,poached eggs on a bed of fried mushrooms and c...,15,45,2,"[poach, egg, bed, fri, mushroom, countri, whit..."
2,pandan chiffon cake,1,45,9,"[pandan, chiffon, cake]"
3,pork chop with honey mustard and apples,17,45,4,"[pork, chop, honey, mustard, appl]"
4,beet gnocchi with steak and brown butter sauce,12,45,4,"[beet, gnocchi, steak, brown, butter, sauc]"
...,...,...,...,...,...
4300,yakitori glaze,1,45,4,"[yakitori, glaze]"
4301,pan seared honey glazed salmon with collard gr...,63,30,2,"[pan, sear, honey, glaze, salmon, collard, green]"
4302,white chocolate cheesecake with raspberries,2,45,16,"[white, chocol, cheesecak, raspberri]"
4303,fish pie with fresh and smoked salmon,90,45,2,"[fish, pie, fresh, smoke, salmon]"


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
text = tex['title']
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# print(vectorizer.vocabulary_)
vector = vectorizer.transform(text)
# resultado
print(vector.shape)#num de docs por number of words
print(type(vector))
print(vector.toarray())
### Correct typos
co=tex["cols"]
listatodo=[]
for i in co:
    for j in i:
        if j not in listatodo:
            listatodo.append(j)
#print(listatodo)
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2))
counts = ngram_vectorizer.fit_transform(listatodo)
#ngram_vectorizer.get_feature_names()
counts.toarray().astype(int)

In [268]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = tex['title']
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
# print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
# print(vector.shape)
vector = vectorizer.transform(text)
# resultado
print(vector.shape)#num de doscs por numero de palabras
print(type(vector))
v=vector.toarray()

[7.57600521 8.26915239 8.26915239 ... 8.26915239 5.43593905 8.6746175 ]
(4305, 2333)
<class 'scipy.sparse.csr.csr_matrix'>


## DIVISION TRAIN - TEST

In [269]:
X_train, X_eval, y_train, y_eval = train_test_split(v, data['readyInMinutes'], test_size=0.3, random_state=42)

In [270]:
X_train, X_eval, y_train, y_eval = train_test_split(v, data['readyInMinutes'], test_size=0.3, random_state=42)pocas=X_train
pocasy=y_train
pocast=X_eval
pocasyt=y_eval

# REGRESION METHODS 

## LogisticRegression

In [273]:
from sklearn.linear_model import LogisticRegression
start = time.time()
clf = LogisticRegression()
clf.fit(pocas, pocasy)
pred = clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')

end = time.time()
print(f"Training comleted in {end-start}s.")

Training comleted in 9.559314727783203s.


In [274]:
score

0.011985058446455504

## svm

In [275]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import svm

In [306]:
start = time.time()
#rbf model
clf = svm.SVC()
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorerbf= f1_score(y_eval, pred, average='macro')
scorerbf
end = time.time()
print(f"Training comleted in {end-start}s.")
scorerbf

Training comleted in 90.22060346603394s.


0.05409721870679238

In [281]:
start = time.time()
clf = svm.SVC(C=0.5,kernel='linear')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorelineal = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorelineal

Training comleted in 33.157859563827515s.


0.017099240480861127

In [282]:

start = time.time()
clf = svm.SVC(C=0.5,kernel='linear',class_weight='balanced')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorelineal = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorelineal

Training comleted in 331.6145942211151s.


0.11243298758973122

In [283]:

start = time.time()
clf = svm.SVC(kernel='poly')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorepoly = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorepoly

Training comleted in 352.9163384437561s.


0.12187760005910254

In [284]:
start = time.time()
clf = svm.SVC(kernel='poly',class_weight='balanced')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorepoly = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorepoly

Training comleted in 515.7796738147736s.


0.19364492347270784

## DecisionTreeClassifier

In [278]:

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(pocas, pocasy)
clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
score

Training comleted in 31.630287170410156s.


0.05409721870679238

## perceptron multicapa

In [280]:

start = time.time()
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(pocas, pocasy)
clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
score

Training comleted in 94.48288249969482s.


0.05409721870679238

### REPEATED ( Changing the input data)

In [289]:
tex = data.iloc[:,[3,-4,-2,-1]]
tex

,instructions,healthScore,readyInMinutes,servings
0,you will need a 9 springform pan or a cake ...,3.0,45,10
1,in a frying pan heat up oil then add mushroom...,15.0,45,2
2,preheat the oven to 170c blend the pandan le...,1.0,45,9
3,pre heat your oven to 200c 400f line a roa...,17.0,45,4
4,cooking beets heat oven to 400 degrees wash be...,12.0,45,4
...,...,...,...,...
4300,combine all ingredients in a small saucepan a...,1.0,45,4
4301,instructions collard greens using a non stick ...,63.0,30,2
4302,to make the crumb crust in a small bowl combin...,2.0,45,16
4303,peel potatoes and cut into chunks cook in boi...,90.0,45,2


In [295]:
stop_words=set(stopwords.words('english'))
lista=[]
for fila in tex["instructions"]:
    try:
            fil=fila.lower()
    except : pass
        #eliminar webs y https
    sinwebs=re.sub(r"https\S+|https\S+|www\S+",'',fil,flags=re.MULTILINE)
    #tokenizar
    tok=word_tokenize(sinwebs)
    filaa=[]
    for pal in tok:
        #eliminar stopwords
        if pal not in stop_words:
            if pal.isalnum():
                #stemizar
                z=ps.stem(pal)
                #lematizar
                zz=lemmatizer.lemmatize(z)
                filaa.append(zz)
    lista.append(filaa) 
tex['cols1']=lista



<ipython-input-295-96b7fc8e0212>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tex['cols1']=lista


In [301]:
X_train, X_eval, y_train, y_eval = train_test_split(v, data['readyInMinutes'], test_size=0.3, random_state=42)
pocas=X_train
pocasy=y_train
pocast=X_eval
pocasyt=y_eval

In [302]:
from sklearn.linear_model import LogisticRegression
start = time.time()
clf = LogisticRegression()
clf.fit(pocas, pocasy)
pred = clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')

end = time.time()
print(f"Training comleted in {end-start}s.")

Training comleted in 9.719621181488037s.


In [305]:
start = time.time()
#rbf model
clf = svm.SVC()
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorerbf= f1_score(y_eval, pred, average='macro')
print(scorerbf)
scorerbf
end = time.time()
print(f"Training comleted in {end-start}s.")
score
print(score)
start = time.time()
clf = svm.SVC(C=0.5,kernel='linear')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorelineal = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorelineal
print(scorelineal)
start = time.time()
clf = svm.SVC(C=0.5,kernel='linear',class_weight='balanced')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorelineal = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorelineal
print(scorelineal)

start = time.time()
clf = svm.SVC(kernel='poly')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorepoly = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorepoly
print(scorepoly)

start = time.time()
clf = svm.SVC(kernel='poly',class_weight='balanced')
clf.fit(X_train,y_train)
pred = clf.predict(X_eval)
scorepoly = f1_score(y_eval, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
scorepoly
print(scorepoly)

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(pocas, pocasy)
clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
score
print(score)
start = time.time()
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(pocas, pocasy)
clf.predict(pocast)
score = f1_score(pocasyt, pred, average='macro')
end = time.time()
print(f"Training comleted in {end-start}s.")
score
print(score)

0.05409721870679238
Training comleted in 88.35710620880127s.
0.19364492347270784
Training comleted in 33.12643003463745s.
0.017099240480861127
Training comleted in 330.2898299694061s.
0.11243298758973122
Training comleted in 358.275545835495s.
0.12187760005910254
Training comleted in 525.1029150485992s.
0.19364492347270784
Training comleted in 557.2320039272308s.
0.19364492347270784
Training comleted in 92.99646496772766s.
0.19364492347270784


In [71]:
! pip install transformers

In [72]:
from transformers import BertTokenizer, BertForMaskedLM
df=pd.read_csv('C:/Users/sarab/Downloads/cleaneddata.csv')
print(df.sample(5))

## create label and sentence list
sentences = df.title.values #TEXTO

#check distribution of data based on labels
print("Distribution of data based on labels: ",df.readyInMinutes.value_counts())#CLASSIFICACION

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 512

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]
readyInMinutes = df.readyInMinutes.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

      Unnamed: 0                                              title  \
10            10                                 broiled crab cakes   
3580          75                                little italy burger   
3123          19                      carrot raisin pineapple salad   
2640          73                      cheesy mexican turkey burgers   
1530          30  15 minute tamari marinated steak salad [paleo ...   

                                                summary  \
10    broiled crab cakes takes around approximately ...   
3580  little italy burger might be just the american...   
3123  carrot raisin pineapple salad is a gluten free...   
2640  cheesy mexican turkey burgers is a gluten free...   
1530  15 minute tamari marinated steak salad [paleo ...   

                                           instructions  \
10    place all of the ingredients  except the butte...   
3580  heat the grill to high  divide the ground beef...   
3123   prepare ingredients  mix together 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\sarab\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2263: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Actual sentence before tokenization:  pandan chiffon cake
Encoded Input from dataset:  [101, 25462, 2078, 9610, 4246, 2239, 9850, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [96]:
stop_words=set(stopwords.words('english'))
lista=[]
for fila in tex["title"]:
    fil=fila.lower()
    #eliminar webs y https
    sinwebs=re.sub(r"https\S+|https\S+|www\S+",'',fil,flags=re.MULTILINE)
    #tokenizar
    tok=word_tokenize(sinwebs)
    filaa=[]
    for pal in tok:
        #eliminar stopwords
        if pal not in stop_words:
            if pal.isalnum():
                #stemizar
                z=ps.stem(pal)
                #lematizar
                zz=lemmatizer.lemmatize(z)
                filaa.append(zz)
    lista.append(filaa) 
tex['cols']=lista

<ipython-input-96-a7064eb2ddad>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tex['cols']=lista


In [97]:
tex

,title,healthScore,readyInMinutes,servings,cols
0,orange fig teacake with caramel glaze,3,45,10,"[orang, fig, teacak, caramel, glaze]"
1,poached eggs on a bed of fried mushrooms and c...,15,45,2,"[poach, egg, bed, fri, mushroom, countri, whit..."
2,pandan chiffon cake,1,45,9,"[pandan, chiffon, cake]"
3,pork chop with honey mustard and apples,17,45,4,"[pork, chop, honey, mustard, appl]"
4,beet gnocchi with steak and brown butter sauce,12,45,4,"[beet, gnocchi, steak, brown, butter, sauc]"
...,...,...,...,...,...
4300,yakitori glaze,1,45,4,"[yakitori, glaze]"
4301,pan seared honey glazed salmon with collard gr...,63,30,2,"[pan, sear, honey, glaze, salmon, collard, green]"
4302,white chocolate cheesecake with raspberries,2,45,16,"[white, chocol, cheesecak, raspberri]"
4303,fish pie with fresh and smoked salmon,90,45,2,"[fish, pie, fresh, smoke, salmon]"


Select variables, 0:5 text , -4: num

In [84]:
t = data.iloc[:,[0,1,2,3,4,5,-4,-3,-2,-1]]

In [272]:
t1 = t
for i in list(t.columns[0:6]):
    stop_words=set(stopwords.words('english'))
    lista=[]
    for fila in t[i]:
        try:
            fil=fila.lower()
        except : pass
        #eliminar webs y https
        sinwebs=re.sub(r"https\S+|https\S+|www\S+",'',fil,flags=re.MULTILINE)
        #tokenizar
        tok=word_tokenize(sinwebs)
        filaa=[]
        for pal in tok:
            #eliminar stopwords
            if pal not in stop_words:
                if pal.isalnum():
                    #stemizar
                    z=ps.stem(pal)
                    #lematizar
                    zz=lemmatizer.lemmatize(z)
                    filaa.append(zz)
        lista.append(filaa) 
    t[i+'lema']=lista

<ipython-input-272-87a3aaa77e76>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t[i+'lema']=lista


In [122]:
t

,title,summary,instructions,ingredients,ingredient types,diets,healthScore,pricePerServing,readyInMinutes,servings,titlelema,summarylema,instructionslema,ingredientslema,ingredient typeslema,dietslema
0,orange fig teacake with caramel glaze,orange fig teacake with caramel glaze is a veg...,you will need a 9 springform pan or a cake ...,ap flour; baking powder; cardamom; eggs; fresh...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,3,75.55,45,10,"[orang, fig, teacak, caramel, glaze]","[orang, fig, teacak, caramel, glaze, vegetaria...","[need, 9, springform, pan, cake, pan, least, 2...","[ap, flour, bake, powder, cardamom, egg, fresh...","[beverag, milk, egg, dairi, spice, season, bak...","[lacto, ovo, vegetarian]"
1,poached eggs on a bed of fried mushrooms and c...,poached eggs on a bed of fried mushrooms and c...,in a frying pan heat up oil then add mushroom...,bread; butter; eggs; eggs; mushrooms; oil; sal...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,15,147.70,45,2,"[poach, egg, bed, fri, mushroom, countri, whit...","[poach, egg, bed, fri, mushroom, countri, whit...","[fri, pan, heat, oil, add, mushroom, saut, 5, ...","[bread, butter, egg, egg, mushroom, oil, salt,...","[beverag, milk, egg, dairi, spice, season, oil...","[lacto, ovo, vegetarian]"
2,pandan chiffon cake,for 26 cents per serving this recipe covers ...,preheat the oven to 170c blend the pandan le...,all purpose flour; bay leaves; coconut milk; c...,Ethnic Foods Produce Spices and Seasonings Bev...,dairy free; lacto ovo vegetarian,1,26.06,45,9,"[pandan, chiffon, cake]","[26, cent, per, serv, recip, cover, 3, daili, ...","[preheat, oven, 170c, blend, pandan, leav, wat...","[purpos, flour, bay, leav, coconut, milk, corn...","[ethnic, food, produc, spice, season, beverag,...","[dairi, free, lacto, ovo, vegetarian]"
3,pork chop with honey mustard and apples,pork chop with honey mustard and apples might...,pre heat your oven to 200c 400f line a roa...,apples; dijon mustard; garlic cloves; honey; j...,Meat Spices and Seasonings Condiments Oil Vine...,gluten free; dairy free; paleolithic; primal,17,242.23,45,4,"[pork, chop, honey, mustard, appl]","[pork, chop, honey, mustard, appl, might, main...","[pre, heat, oven, 200c, 400f, line, roast, tin...","[appl, dijon, mustard, garlic, clove, honey, j...","[meat, spice, season, condiment, oil, vinegar,...","[gluten, free, dairi, free, paleolith, primal]"
4,beet gnocchi with steak and brown butter sauce,the recipe beet gnocchi with steak and brown b...,cooking beets heat oven to 400 degrees wash be...,gnocchi; beets; olive oil; s p; goat cheese; r...,Produce Spices and Seasonings Meat Spices and ...,,12,417.69,45,4,"[beet, gnocchi, steak, brown, butter, sauc]","[recip, beet, gnocchi, steak, brown, butter, s...","[cook, beet, heat, oven, 400, degre, wash, bee...","[gnocchi, beet, oliv, oil, p, goat, chees, ric...","[produc, spice, season, meat, spice, season, m...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,yakitori glaze,yakitori glaze might be just the japanese reci...,combine all ingredients in a small saucepan a...,butter; egg yolks; flour; orange zest; poppy s...,Milk Eggs Other Dairy Spices and Seasonings Ba...,lacto ovo vegetarian,1,193.70,45,4,"[yakitori, glaze]","[yakitori, glaze, might, japanes, recip, searc...","[combin, ingredi, small, saucepan, bring, boil...","[butter, egg, yolk, flour, orang, zest, poppi,...","[milk, egg, dairi, spice, season, bake, spice,...","[lacto, ovo, vegetarian]"
4301,pan seared honey glazed salmon with collard gr...,pan seared honey glazed salmon with collard gr...,instructions collard greens using a non stick ...,unbaked pie crust; shredded cheddar cheese; ja...,Milk Eggs Other Dairy Meat Spices and Seasonin...,,63,493.63,30,2,"[pan, sear, honey, glaze, salmon, collard, green]","[pan, sear, honey, glaze, salmon, collard, gre...","[instruct, collard, green, use, non, stick, pa...","[unbak, pie, crust, shr

In [258]:
 dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__ignoreds',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__numpys',
 '__recursive_saveloads',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__scipys',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 '

http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/ old version , for modifications : https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4#3-index2word-and-index2entity-attribute-is-now-index_to_key

# BERT

In [148]:
# !git clone -b master https://github.com/charles9n/bert-sklearn
# !cd bert-sklearn; pip install .
import os


In [154]:
from bert_sklearn import BertClassifier
from bert_sklearn import load_model
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [156]:
X_train, X_eval, y_train, y_eval = train_test_split(t['titlelema'], tex['healthScore'], test_size=0.3, random_state=42)

In [159]:
X_train, y_train


(1627                                [dolsot, bibimbap]
 258              [crockpot, spaghetti, pesto, meatbal]
 3280                               [autumn, cheesecak]
 386                              [almond, joy, cupcak]
 847     [spaghetti, squash, tomato, basil, meat, sauc]
                              ...                      
 3444                    [chocol, chip, protein, cooki]
 466                  [ham, mushroom, sauerkraut, soup]
 3092                      [dri, prawn, yee, mee, soup]
 3772                                    [bbq, chicken]
 860                           [homemad, anim, cracker]
 Name: titlelema, Length: 3013, dtype: object,
 1627    67
 258      8
 3280     1
 386      1
 847     65
         ..
 3444     1
 466     16
 3092     5
 3772     8
 860      2
 Name: healthScore, Length: 3013, dtype: int32)

In [160]:
model = BertClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X_test, y_test) # loss, accuracy
score = f1_score(y_test, y_pred, average='macro')

Building sklearn text classifier...


100%|██████████| 231508/231508 [00:00<00:00, 618502.51B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 108523.07B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2712, validation data size: 301


...


Validating: 100%|██████████| 38/38 [02:31<00:00,  3.98s/it]

Epoch 2, Train loss: 3.7036, Val loss: 3.7100, Val accy: 10.63%



Validating: 100%|██████████| 38/38 [02:38<00:00,  4.17s/it]

Epoch 3, Train loss: 3.5639, Val loss: 3.6729, Val accy: 9.97%


NameError: name 'X_test' is not defined

In [175]:
savefile='./mymodeltitlehealth.bin'
model.save(savefile)
new_model = load_model(savefile)

Loading model from ./mymodeltitlehealth.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [173]:
y_pred = model.predict(X_eval)
# model.score(X_eval, y_eval) # loss, accuracy
score = f1_score(y_eval, y_pred, average='macro')

Predicting: 100%|██████████| 162/162 [10:22<00:00,  3.84s/it]


In [174]:
score

0.007650121001169413

In [176]:
X_train, X_eval, y_train, y_eval = train_test_split(t['instructionslema'], tex['servings'], test_size=0.3, random_state=42)

In [177]:
model = BertClassifier() 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.score(X_test, y_test) # loss, accuracy
score = f1_score(y_test, y_pred, average='macro')

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2712, validation data size: 301


C:\Users\sarab\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validating: 100%|██████████| 38/38 [02:34<00:00,  4.07s/it]

Epoch 1, Train loss: 2.7601, Val loss: 2.4864, Val accy: 27.24%



Validating: 100%|██████████| 38/38 [02:16<00:00,  3.59s/it]

Epoch 2, Train loss: 2.4627, Val loss: 2.4690, Val accy: 27.24%



Validating: 100%|██████████| 38/38 [02:13<00:00,  3.53s/it]

Epoch 3, Train loss: 2.4488, Val loss: 2.4676, Val accy: 27.24%


NameError: name 'X_test' is not defined

In [178]:
savefile='./mymodelinstructionsservings.bin'
model.save(savefile)
new_model = load_model(savefile)

Loading model from ./mymodelinstructionsservings.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [179]:
y_pred = model.predict(X_eval)
# model.score(X_eval, y_eval) # loss, accuracy
score = f1_score(y_eval, y_pred, average='macro')

Predicting: 100%|██████████| 162/162 [08:43<00:00,  3.23s/it]


In [180]:
score

0.010931009712982916

In [181]:
X_train, X_eval, y_train, y_eval = train_test_split(t['instructionslema'], tex['readyInMinutes'], test_size=0.3, random_state=42)

In [182]:
model = BertClassifier() 
model.fit(X_train, y_train)

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2712, validation data size: 301


C:\Users\sarab\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validating: 100%|██████████| 38/38 [02:41<00:00,  4.24s/it]

Epoch 1, Train loss: 2.1877, Val loss: 1.7508, Val accy: 70.76%



Validating: 100%|██████████| 38/38 [02:32<00:00,  4.01s/it]

Epoch 2, Train loss: 1.5831, Val loss: 1.6915, Val accy: 70.76%



Validating: 100%|██████████| 38/38 [02:26<00:00,  3.86s/it]

Epoch 3, Train loss: 1.5598, Val loss: 1.6818, Val accy: 70.76%


BertClassifier(do_lower_case=True,
               label_list=array([   2,    3,    4,    5,    6,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   20,   22,   23,   24,   25,   27,
         28,   29,   30,   31,   32,   33,   34,   35,   36,   37,   38,
         40,   41,   44,   45,   46,   47,   50,   55,   57,   60,   64,
         65,   70,   74,   75,   77,   80,   82,   84,   85,   90,   95,
        100,  105,  110,  115,  120,  125,  130,  135,  140,  150,  160,
        165,  170,  180,  185,  200,  205,  210,  215,  220,  230,  240,
        250,  255,  260,  270,  275,  300,  310,  315,  350,  360,  370,
        380,  390,  400,  420,  425,  450,  465,  485,  490,  495,  500,
        520,  540,  615,  660,  845,  910,  980, 1440, 1460, 1500, 3595],
      dtype=int64))

In [184]:
savefile='./mymodelinstructionsinminutes.bin'
model.save(savefile)
new_model = load_model(savefile)

Loading model from ./mymodelinstructionsinminutes.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [183]:
y_pred = model.predict(X_eval)
f1_score(y_eval, y_pred, average='macro')

Predicting: 100%|██████████| 162/162 [08:33<00:00,  3.17s/it]


0.011545806321925724

In [185]:
X_train, X_eval, y_train, y_eval = train_test_split(t['ingredientslema'], tex['readyInMinutes'], test_size=0.3, random_state=42)

In [186]:
model = BertClassifier() 
model.fit(X_train, y_train)

Building sklearn text classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 2712, validation data size: 301


C:\Users\sarab\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Validating: 100%|██████████| 38/38 [02:36<00:00,  4.13s/it]

Epoch 1, Train loss: 2.2164, Val loss: 1.7440, Val accy: 70.76%



Validating: 100%|██████████| 38/38 [03:02<00:00,  4.81s/it]

Epoch 2, Train loss: 1.5825, Val loss: 1.6984, Val accy: 70.76%



Validating: 100%|██████████| 38/38 [02:37<00:00,  4.13s/it]

Epoch 3, Train loss: 1.5666, Val loss: 1.6990, Val accy: 70.76%


BertClassifier(do_lower_case=True,
               label_list=array([   2,    3,    4,    5,    6,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   20,   22,   23,   24,   25,   27,
         28,   29,   30,   31,   32,   33,   34,   35,   36,   37,   38,
         40,   41,   44,   45,   46,   47,   50,   55,   57,   60,   64,
         65,   70,   74,   75,   77,   80,   82,   84,   85,   90,   95,
        100,  105,  110,  115,  120,  125,  130,  135,  140,  150,  160,
        165,  170,  180,  185,  200,  205,  210,  215,  220,  230,  240,
        250,  255,  260,  270,  275,  300,  310,  315,  350,  360,  370,
        380,  390,  400,  420,  425,  450,  465,  485,  490,  495,  500,
        520,  540,  615,  660,  845,  910,  980, 1440, 1460, 1500, 3595],
      dtype=int64))

In [187]:
savefile='./mymodelingredientsinminutes.bin'
model.save(savefile)
new_model = load_model(savefile)

Loading model from ./mymodelingredientsinminutes.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


In [188]:
y_pred = model.predict(X_eval)
f1_score(y_eval, y_pred, average='macro')

Predicting: 100%|██████████| 162/162 [09:49<00:00,  3.64s/it]


0.011545806321925724

In [211]:
y_pred = model.predict_proba(X_eval)

C:\Users\sarab\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Predicting: 100%|██████████| 162/162 [10:07<00:00,  3.75s/it]


In [194]:
np.amax(y_pred)

0.7330921

In [202]:
y_pred = np.array(y_pred)

In [206]:
len(y_pred)
y_pred[0]

array([7.0191920e-04, 2.7609745e-04, 5.5767712e-04, 6.4984593e-03,
       5.1068945e-04, 3.4127527e-04, 5.5356970e-04, 1.6071022e-02,
       7.2688347e-04, 3.4052361e-04, 5.7525019e-04, 4.6103194e-04,
       1.5992651e-02, 7.4952183e-04, 4.2284644e-04, 1.1205886e-03,
       1.3612257e-02, 9.1865566e-04, 5.6325755e-04, 2.7149479e-04,
       1.5442854e-02, 7.8833033e-04, 2.4012367e-03, 5.7630584e-04,
       3.8512114e-02, 5.2325480e-04, 5.5834127e-04, 3.9455103e-04,
       3.4985199e-04, 1.2640856e-02, 5.2199204e-04, 4.9945037e-04,
       5.5524922e-04, 1.4081030e-02, 9.0136984e-04, 4.8345231e-04,
       7.2309226e-01, 3.6885243e-04, 4.9275602e-04, 8.4094619e-03,
       7.9541169e-03, 4.0869438e-04, 2.2682641e-02, 5.3224887e-04,
       4.5949784e-03, 5.0346078e-03, 5.1630248e-04, 1.3578891e-02,
       7.5623940e-04, 3.6186709e-03, 5.5201375e-04, 6.0606765e-04,
       6.8384287e-04, 7.4105039e-03, 1.3131786e-03, 1.0172301e-03,
       1.5008138e-03, 1.0285012e-03, 4.4902868e-04, 2.5206443e

In [209]:
y_prob = []
for i in y_pred : 
    y_prob.append(max(i))

In [210]:
df_describe = pd.DataFrame(y_prob)
df_describe.describe()

,0
count,1292.000000
mean,0.721892
std,0.005796
min,0.704161
25%,0.718619
50%,0.722842
75%,0.725816
max,0.733092


The mean value of the higher probability prediction it's around 70% , quite high considering the f-score it's really low, there is not predicting the correct value